# | Project Introduction |

--Project Introduction--
## Stages
--Stages Overview--
1. **Data overview:** there are multiple datasets, that are not ready to go as is.  I will be utilizing arguments that will allow me to properly access the data.
2. **Preprocessing:** Cleaning and preprocessing the data to ensure accuracy.
3. **Analysis:** Calculating the revenue generated by each plan based on the users' monthly consumption and overage charges.

# Stage 1: Data overview 
- Here I will open the datasets for use in the project. First though, I must import all necessary python libraries. This will allow me to use multiple different helpful methods.

In [1]:
# import all libraries here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

print ("Libraries working as they should")

Libraries working as they should


### Reading in DataFrame: | vehicles_us |

Brief description of the different columns:<br>
 - **price:**&nbsp;&nbsp;The selling price of the vehicle in USD.<br><br>
 - **model_year:** model_year: The year in which the vehicle model was manufactured (float). Missing values indicate vehicles with unknown model years.<br><br>
 - **model:** The specific make and model of the vehicle (e.g., "bmw x5" or "ford f-150").<br><br>
 - **condition:** The condition of the vehicle (e.g., "excellent," "good," "fair," etc.).<br><br>
 - **cylinders:** The number of engine cylinders in the vehicle, which often affects engine power and fuel consumption (float). Missing values indicate unknown cylinder counts.<br><br>
 - **fuel:** The type of fuel the vehicle uses (e.g., "gas," "diesel," "electric," etc.).<br><br>
 - **odometer:** The distance traveled by the vehicle in miles. Missing values indicate odometer readings not provided by the seller.<br><br>
 - **transmission:** The type of transmission in the vehicle (e.g., "automatic" or "manual").<br><br>
- **type:** The body style or type of the vehicle (e.g., "sedan," "SUV," "pickup").<br><br>
- **paint_color:** The exterior color of the vehicle (e.g., "black," "white," "blue"). Missing values represent cars with unknown or unspecified colors.<br><br>
- **is_4wd:** Indicates whether the vehicle has 4-wheel drive (1 = yes, NaN or 0 = no).<br><br>
- **date_posted:** The date when the vehicle listing was posted online (datetime).<br><br>
- **days_listed:** The number of days the vehicle listing has been active before being sold or removed (integer).<br><br>

In [2]:
vehicles_us = pd.read_csv(r'C:\Users\sicol\whipdata\whipdata_r\vehicles_us.csv')
vehicles_us.sample(5)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
37922,6995,2003.0,ford mustang,excellent,NaN,gas,NaN,automatic,convertible,NaN,NaN,2018-05-22,1
2165,2500,1996.0,jeep cherokee,good,6.0,gas,192000.0,automatic,SUV,green,1.0,2018-12-09,32
37838,4500,2008.0,chevrolet silverado,good,8.0,gas,336000.0,automatic,truck,white,1.0,2018-07-12,57
10305,28000,2018.0,toyota camry,new,6.0,gas,1277.0,automatic,sedan,silver,NaN,2018-11-30,36
12232,8800,NaN,honda cr-v,excellent,4.0,gas,133516.0,automatic,wagon,red,1.0,2018-05-11,45


## Stage 1 Conclusion
**This was a detailed overview of the dataset.  The data was successully loaded into a DataFrame for further exploration.  This sets the stage for further analysis.**

# Stage 2: Preprocessing

### Identifying and handling missing values
<b>From the data shown in the sample above, and as we wil analyze belowe, we can see a few things that need to be corrected:<b/>
- There are missing values for 'model_year. For effective data handling, I will be filling these in with the median values.
    - The same will be done for 'cylinders'.
    - The same will be done for 'odometer'.
- There are missing values for 'paint_color'.  Since these ar object type, I will just replce these with 'unknown'.
- There are missing values for 'is_4wd'.  Since these likely mean that the vehicle is not 4 wheel drive, I will repace them with '0'.

In [3]:
vehicles_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [4]:
vehicles_us.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


In [5]:
# Check for missing data
missing_data = vehicles_us.isnull().sum()
missing_percentage = (missing_data / len(vehicles_us)) * 100

# Combine into a seperate dataframe for better readability
missing_df = pd.DataFrame({
    'Missing Values': missing_data,
    'Percentage (%)': missing_percentage
})

# Display missing values
missing_df

,Missing Values,Percentage (%)
price,0,0.000000
model_year,3619,7.023775
model,0,0.000000
condition,0,0.000000
cylinders,5260,10.208637
fuel,0,0.000000
odometer,7892,15.316836
transmission,0,0.000000
type,0,0.000000
paint_color,9267,17.985444


In [6]:
# Fill numeric columns with median values
vehicles_us['model_year'] = vehicles_us['model_year'].fillna(vehicles_us['model_year'].median())
vehicles_us['cylinders'] = vehicles_us['cylinders'].fillna(vehicles_us['cylinders'].median())
vehicles_us['odometer'] = vehicles_us['odometer'].fillna(vehicles_us['odometer'].median())

# Fill categorical columns with 'unknown' or 0
vehicles_us['paint_color'] = vehicles_us['paint_color'].fillna('unknown')
vehicles_us['is_4wd'] = vehicles_us['is_4wd'].fillna(0)

# Verify that missing data has been handled
print(vehicles_us.isnull().sum())

price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64


In [7]:
vehicles_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [8]:
vehicles_us.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,51525.000000,51525.000000,51525.000000,51525.000000,51525.00000
mean,12132.464920,2009.838234,6.112450,115162.352179,0.496303,39.55476
std,10040.803015,6.065836,1.573784,59909.264385,0.499991,28.20427
min,1.000000,1908.000000,3.000000,0.000000,0.000000,0.00000
25%,5000.000000,2007.000000,4.000000,79181.000000,0.000000,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,0.000000,33.00000
75%,16839.000000,2014.000000,8.000000,146541.000000,1.000000,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.000000,271.00000


### Checking for duplicated values

In [9]:
# Check for fully duplicate rows
duplicate_rows = vehicles_us.duplicated().sum()
f"There are {duplicate_rows} fully duplicate rows."

'There are 0 fully duplicate rows.'

## Stage 2 Conclusion
**The dataset has been successfully cleaned by handling missing values and checking for duplicates. Missing values in 'model_year', 'cylinders', and 'odometer' were filled with their medians, while 'paint_color' was filled with "unknown" and 'is_4wd' with 0. After the cleanup, all missing values were resolved, and it was confirmed there are no fully duplicate rows. The dataset is now complete, with accurate statistics and ready for further analysis.**

# Stage 3: Analysis
- Stage introduction

### Identifying unique values in each column

In [10]:
# Display summary statistics
print(vehicles_us.describe())

# Identify unique values in categorical columns
categorical_columns = ['model', 'condition', 'fuel', 'transmission', 'type', 'paint_color']
for col in categorical_columns:
    print(f"Unique values in {col}: {vehicles_us[col].nunique()}")

               price    model_year     cylinders       odometer        is_4wd  \
count   51525.000000  51525.000000  51525.000000   51525.000000  51525.000000   
mean    12132.464920   2009.838234      6.112450  115162.352179      0.496303   
std     10040.803015      6.065836      1.573784   59909.264385      0.499991   
min         1.000000   1908.000000      3.000000       0.000000      0.000000   
25%      5000.000000   2007.000000      4.000000   79181.000000      0.000000   
50%      9000.000000   2011.000000      6.000000  113000.000000      0.000000   
75%     16839.000000   2014.000000      8.000000  146541.000000      1.000000   
max    375000.000000   2019.000000     12.000000  990000.000000      1.000000   

       days_listed  
count  51525.00000  
mean      39.55476  
std       28.20427  
min        0.00000  
25%       19.00000  
50%       33.00000  
75%       53.00000  
max      271.00000  
Unique values in model: 100
Unique values in condition: 6
Unique values in fuel: 

### Histogram of 'price'
This shows the distribution of car prices:

In [11]:
# Filter dataset to focus on prices under $100K
filtered_data = vehicles_us[vehicles_us['price'] < 100000]

fig = px.histogram(filtered_data, x='price', nbins=40, title='Distribution of Car Prices (Below $100K)')
fig.update_layout(
    xaxis_title='Car Price (USD)',
    yaxis_title='Number of Listings',
    title_font_size=20,
    title_x=0.5,  # Center the title
    showlegend=True  # Enable the legend
)

# Add vertical lines for mean and median
mean_price = filtered_data['price'].mean()
median_price = filtered_data['price'].median()

fig.add_vline(x=mean_price, line_width=2, line_dash="dash", line_color="black")
fig.add_vline(x=median_price, line_width=2, line_dash="dash", line_color="red")

# Add dummy traces for the legend using go.Scatter
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color="red", dash="dash"),
                         name="Median Price"))
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color="black", dash="dash"),
                         name="Mean Price"))

fig.show()

### Histogram of 'odometer'
This displays the mileage distribution:

In [12]:
# Filter dataset for odometer readings less than 400,000 miles
filtered_odometer_data = vehicles_us[vehicles_us['odometer'] < 400000]

# Create the histogram
fig = px.histogram(filtered_odometer_data, x='odometer', nbins=50, title='Distribution of Odometer Readings (Below 500K Miles)')
fig.update_layout(
    xaxis_title='Odometer (miles)',
    yaxis_title='Number of Listings',
    title_font_size=20,
    title_x=0.5,  # Center the title
    showlegend=True  # Enable the legend
)
fig.update_traces(marker_color='blue', opacity=0.7)  # Change color and transparency

# Calculate mean and median
mean_odometer = filtered_odometer_data['odometer'].mean()
median_odometer = filtered_odometer_data['odometer'].median()

# Add vertical lines for mean and median
fig.add_vline(x=mean_odometer, line_width=2, line_dash="dash", line_color="black")
fig.add_vline(x=median_odometer, line_width=2, line_dash="dash", line_color="red")

# Add dummy traces for the legend
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color="red", dash="dash"),
                         name="Median Odometer"))

fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color="black", dash="dash"),
                         name="Mean Odometer"))

# Display the histogram
fig.show()

### Scatterplot of 'price' vs 'odometer'
This will explore the relationship between price and mileage:

In [13]:
# Filter dataset to focus on prices under $100K and mileage under 400,000
filtered_data = vehicles_us[(vehicles_us['odometer'] < 400000) & (vehicles_us['price'] < 100000)]

fig = px.scatter(
    filtered_data,
    x='odometer',
    y='price',
    color='condition',  # Color points by condition
    title='Price vs Odometer by Condition',
    labels={'odometer': 'Odometer (miles)', 'price': 'Price (USD)'},
    hover_data=['model']  # Add 'model' to hover data for more interactivity
)

# Adjust transparency and size for better visibility
fig.update_traces(marker=dict(opacity=0.6, size=5))

# Improve layout and styling
fig.update_layout(
    xaxis_title='Odometer (miles)',
    yaxis_title='Price (USD)',
    title_font_size=20,
    title_x=0.5,  # Center the title
    template='plotly_white'  # Use a cleaner template
)

# Display the scatterplot
fig.show()

### Scatterplot of 'price' vs 'model_year'
This will visualize how vehicle age impact price:

In [14]:
import datetime

# Calculate car age and filter vehicles less than 65 years old
filtered_data = filtered_data.copy() # To avoid warning
current_year = datetime.datetime.now().year
filtered_data['age'] = current_year - filtered_data['model_year']
filtered_data = filtered_data[filtered_data['age'] < 65]

# Create the scatterplot with vehicle year on the x-axis
fig = px.scatter(
    filtered_data,
    x='model_year',
    y='price',
    color='condition',  # Color points by condition
    title='Price vs Vehicle Year by Condition',
    labels={'model_year': 'Vehicle Year', 'price': 'Price (USD)'},
    hover_data=['model', 'model_year']  # Add 'model' and 'vehicle year' for interactivity
)

# Adjust transparency and size for better visibility
fig.update_traces(marker=dict(opacity=0.6, size=5))

# Improve layout and styling
fig.update_layout(
    xaxis_title='Vehicle Year',
    yaxis_title='Price (USD)',
    title_font_size=20,
    title_x=0.5,  # Center the title
    template='plotly_white'  # Use a cleaner template
)

# Display the scatterplot
fig.show()

## Stage 3 Conclusion

**I conducted comprehensive data exploration, analyzing numerical and categorical columns to understand key statistics and unique values. Several visualizations were created, including histograms for price and odometer, along with scatterplots for price vs odometer and price vs model_year. Filters were applied to focus on relevant data, and enhancements like mean/median annotations, legends and hover interactivity were added for clarity and insight. This analysis effectively highlights trends in pricing, mileage, and vehicle age.**